In [1]:
import pandas as pd
import os

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
root_dir = '../'
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

In [3]:
excel_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [4]:
df_dict = COMBINE_harmonizer.load_data_dict(excel_filename, COMBINE_harmonizer.SHEET_MAIN)
df_dict.columns

Index(['Category', 'Subcategory', 'Standardized_VariableNames_Dictionary',
       'type', 'Variable_Description', '#studies w/ this var', 'redcap',
       'comment', 'lower_var', 'var_eq_redcap', 'connect_redcap',
       'snakecase_var', 'PREVENT', 'HEAL', 'LH', 'OC', 'BONBID', 'PBI',
       'Florida', 'MARBLE', 'EMBRACE', 'HELIX', 'NBS - Peeples', 'NINDS'],
      dtype='object')

In [5]:
df_dict[COMBINE_harmonizer.DATA_DICT_CATEGORY].unique()

array(['Pre-intervention', 'Intervention', 'Post-intervention',
       'NICU Discharge'], dtype=object)

## 02. duplicated columns

In [6]:
df_dict_groupby = df_dict.groupby([COMBINE_harmonizer.DATA_DICT_VAR_NAME], as_index=False).agg(_count=(COMBINE_harmonizer.DATA_DICT_VAR_NAME, 'count'))
is_invalid = df_dict_groupby['_count'] > 1
df_dict_groupby[is_invalid]

,Standardized_VariableNames_Dictionary,_count
68,MRIDate,4
210,MRITime,3
348,birthDate,2
476,center,2
1307,subjectID,3


### 02.1 check the index of the duplicated columns. 

to update in `utils_values.build_variable_order_map`.

In [7]:
invalid_columns = list(df_dict_groupby[is_invalid][COMBINE_harmonizer.DATA_DICT_VAR_NAME]) + ['birthTime']

is_invalid_df_dict = df_dict[COMBINE_harmonizer.DATA_DICT_VAR_NAME].isin(invalid_columns)
columns = [COMBINE_harmonizer.DATA_DICT_VAR_NAME, 'type']
df_dict[is_invalid_df_dict][columns].sort_values(by=[COMBINE_harmonizer.DATA_DICT_VAR_NAME])

,Standardized_VariableNames_Dictionary,type
908,MRIDate,date
916,MRIDate,date
927,MRIDate,date
1072,MRIDate,date
909,MRITime,time
917,MRITime,time
1073,MRITime,time
3,birthDate,date
177,birthDate,date
178,birthTime,time
